In [ ]:
# import torch
# print("PyTorch built with CUDA:", torch.version.cuda)

# print("GPU compute capability:", torch.cuda.get_device_capability(0) if torch.cuda.is_available() else "No GPU detected")
# print("GPU name:", torch.cuda.get_device_name(0) if torch.cuda.is_available() else "No GPU detected")

# print("CUDA runtime version:", torch.version.cuda)
# print("Is CUDA available?", torch.cuda.is_available())


c:\Python_Programs\SatSure\mlops-densenet-optimization\venv\Lib\site-packages\torch\cuda\__init__.py:63: FutureWarning: The pynvml package is deprecated. Please install nvidia-ml-py instead. If you did not install pynvml directly, please report this to the maintainers of the package that installed pynvml for you.
  import pynvml  # type: ignore[import]


PyTorch built with CUDA: 12.6
GPU compute capability: (7, 5)
GPU name: NVIDIA GeForce GTX 1650 Ti with Max-Q Design
CUDA runtime version: 12.6
Is CUDA available? True


In [2]:
import torch

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Running benchmarks on device: {DEVICE}")

print(torch.__version__)
print(torch.version.cuda)
print(torch.cuda.is_available())


Running benchmarks on device: cuda
2.8.0+cu126
12.6
True


In [2]:
# from app.utils import get_ram_usage_mb


# import sys
# import os

# # Add project root (two levels up from notebooks/)
# project_root = os.path.abspath(os.path.join(os.getcwd(), "../.."))
# sys.path.append(project_root)

import utils


c:\Python_Programs\SatSure\mlops-densenet-optimization\venv\Lib\site-packages\torch\cuda\__init__.py:63: FutureWarning: The pynvml package is deprecated. Please install nvidia-ml-py instead. If you did not install pynvml directly, please report this to the maintainers of the package that installed pynvml for you.
  import pynvml  # type: ignore[import]


In [4]:
import os
import time
import torch
import torchvision.models as models
from torch.profiler import profile, record_function, ProfilerActivity
from torch.utils.tensorboard import SummaryWriter
import pandas as pd
import psutil
import utils  # our helper functions (in app/utils.py)


# -----------------------------
# CONFIG
# -----------------------------
BATCH_SIZES = [1, 4, 8, 16, 32]
INPUT_SHAPE = (3, 224, 224)
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
RESULTS_CSV = os.path.join("results", "benchmark_results.csv")
LOG_DIR = os.path.join("logs", "tensorboard")


def benchmark_densenet():
    os.makedirs("results", exist_ok=True)
    os.makedirs(LOG_DIR, exist_ok=True)

    writer = SummaryWriter(LOG_DIR)
    results = []

    print(f"Running benchmarks on device: {DEVICE}")

    # -----------------------------
    # Load model
    # -----------------------------
    start_time = time.time()
    model = models.densenet121(weights=models.DenseNet121_Weights.IMAGENET1K_V1).to(DEVICE)  # no pretrained weights
    model.eval()
    model_load_time = (time.time() - start_time) * 1000  # ms

    # -----------------------------
    # Run for each batch size
    # -----------------------------
    for batch_size in BATCH_SIZES:
        print(f"\nBenchmarking batch size = {batch_size}")
        inputs = torch.randn(batch_size, *INPUT_SHAPE).to(DEVICE)

        # warmup
        with torch.no_grad():
            for _ in range(3):
                _ = model(inputs)

        # profiling
        with profile(
            activities=[ProfilerActivity.CPU, ProfilerActivity.CUDA] if DEVICE == "cuda" else [ProfilerActivity.CPU],
            record_shapes=True,
            profile_memory=True,
            with_stack=True
        ) as prof:
            with record_function("model_inference"):
                torch.cuda.synchronize() if DEVICE == "cuda" else None
                start = time.time()
                with torch.no_grad():
                    outputs = model(inputs)
                torch.cuda.synchronize() if DEVICE == "cuda" else None
                end = time.time()

        # latency and throughput
        latency_ms = (end - start) * 1000
        throughput = batch_size / (end - start)

        # system stats
        ram_usage = utils.get_ram_usage_mb()
        vram_usage = utils.get_vram_usage_mb() if DEVICE == "cuda" else 0
        cpu_util = utils.get_cpu_utilization()
        gpu_util = utils.get_gpu_utilization() if DEVICE == "cuda" else 0

        # log to tensorboard
        writer.add_scalar(f"Latency/batch_{batch_size}", latency_ms)
        writer.add_scalar(f"Throughput/batch_{batch_size}", throughput)
        writer.add_scalar(f"RAM_Usage_MB/batch_{batch_size}", ram_usage)
        if DEVICE == "cuda":
            writer.add_scalar(f"VRAM_Usage_MB/batch_{batch_size}", vram_usage)
            writer.add_scalar(f"GPU_Utilization/batch_{batch_size}", gpu_util)

        # append to results
        results.append({
            "model_variant": "densenet121_baseline",
            "batch_size": batch_size,
            "device": DEVICE,
            "ram_usage_mb": ram_usage,
            "vram_usage_mb": vram_usage,
            "cpu_utilization_pct": cpu_util,
            "gpu_utilization_pct": gpu_util,
            "latency_ms": latency_ms,
            "throughput_samples_sec": throughput,
            "accuracy_top1": "NA",  # will be added in Part 2
            "accuracy_top5": "NA",
            "model_size_mb": utils.get_model_size_mb(model),
            "optimization_technique": "baseline",
            "model_load_time_ms": model_load_time
        })

    writer.close()

    # save CSV
    df = pd.DataFrame(results)
    df.to_csv(RESULTS_CSV, index=False)
    print(f"\n✅ Results saved to {RESULTS_CSV}")


if __name__ == "__main__":
    benchmark_densenet()


Running benchmarks on device: cuda

Benchmarking batch size = 1

Benchmarking batch size = 4

Benchmarking batch size = 8

Benchmarking batch size = 16

Benchmarking batch size = 32

✅ Results saved to results\benchmark_results.csv
